In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [31]:
query = """
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?Title ?Revue_wd ?City WHERE {
  ?journal skos:prefLabel ?Title ;
           skos:exactMatch ?Revue_wd ;
           foaf:focus ?focus .
  ?focus <http://rdvocab.info/Elements/placeOfPublication> ?City .
  FILTER (contains(str(?Revue_wd), "wikidata.org")) . 
}
"""

In [32]:
df = pd.read_csv("combined_journal_data_example.csv")

In [33]:
df.head()

,issues_extract,normalized_date,issueArk,Notice,Media URL,Title
0,issues_extract_1,1908-03-21,bpt6k756130m,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise
1,issues_extract_2,1908-03-22,bpt6k7561310,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise
2,issues_extract_3,1908-03-23,bpt6k756132c,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise
3,issues_extract_4,1908-03-24,bpt6k756133r,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise
4,issues_extract_5,1908-03-25,bpt6k7561344,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise


In [34]:
df = df.drop(columns=['issues_extract', 'issueArk'])
df["SPARQL"] = df["Notice"].str.replace('https://gallica.bnf.fr/', 'http://data.bnf.fr/').str.replace('/date', '')

/var/folders/nw/zm4zl9xn4_d_rq3fnglst3n80000gp/T/ipykernel_70462/2064839789.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["SPARQL"] = df["Notice"].str.replace('https://gallica.bnf.fr/', 'http://data.bnf.fr/').str.replace('/date', '')


In [35]:
df.head()

,normalized_date,Notice,Media URL,Title,SPARQL
0,1908-03-21,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451
1,1908-03-22,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451
2,1908-03-23,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451
3,1908-03-24,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451
4,1908-03-25,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451


In [36]:
sparql = SPARQLWrapper("https://data.bnf.fr/sparql")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

In [37]:
sparql = SPARQLWrapper("https://data.bnf.fr/sparql")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)

def execute_query(journal):
    sparql.setQuery(query.replace("?journal", "<"+journal+">"))
    results = sparql.query().convert()
    if "results" in results and "bindings" in results["results"]:
        bindings = results["results"]["bindings"]
        if len(bindings) > 0:
            return {
                "Title": bindings[0]["Title"]["value"],
                "Revue_wd": bindings[0]["Revue_wd"]["value"],
                "City": bindings[0]["City"]["value"]
            }
    return None

In [38]:
result = df["SPARQL"].apply(execute_query)

In [39]:
result_df = pd.DataFrame(result.tolist())

In [40]:
result_df.head()

,Title,Revue_wd,City
0,L'Action française (Paris. 1908),http://wikidata.org/entity/Q496011,Paris
1,L'Action française (Paris. 1908),http://wikidata.org/entity/Q496011,Paris
2,L'Action française (Paris. 1908),http://wikidata.org/entity/Q496011,Paris
3,L'Action française (Paris. 1908),http://wikidata.org/entity/Q496011,Paris
4,L'Action française (Paris. 1908),http://wikidata.org/entity/Q496011,Paris


In [44]:
result_df["Revue_wd"] = result_df["Revue_wd"].str.replace('http://wikidata.org/entity/', '', regex=True)

In [42]:
df = pd.concat([df, result_df], axis=1)

In [43]:
df.head()

,normalized_date,Notice,Media URL,Title,SPARQL,Title,Revue_wd,City
0,1908-03-21,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451,L'Action française (Paris. 1908),Q496011,Paris
1,1908-03-22,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451,L'Action française (Paris. 1908),Q496011,Paris
2,1908-03-23,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451,L'Action française (Paris. 1908),Q496011,Paris
3,1908-03-24,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451,L'Action française (Paris. 1908),Q496011,Paris
4,1908-03-25,https://gallica.bnf.fr/ark:/12148/cb326819451/...,https://gallica.bnf.fr/iiif/ark:/12148/bpt6k75...,Action francaise,http://data.bnf.fr/ark:/12148/cb326819451,L'Action française (Paris. 1908),Q496011,Paris
